In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import theano
import theano.tensor as tt
import arviz as az
from theano import shared
from scipy import stats
import pandas as pd
import math

# Helper functions

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

def normalize(x):
    return x/np.sum(x)

def rmse(pred, actual):
    return math.sqrt(np.mean((pred-actual)**2))

def sort_category(unique, count,n):
    output = np.ones(n) - 1
    for i in range(n):
        for j in range(len(unique)):
            if(i==unique[j]):
                output[i]=count[j]
    return normalize(output) 

def perturb(x,noise=.001):
    return tt.switch(tt.eq(x,1),1-noise, tt.switch(tt.eq(x,0), noise, x))

def compute_ci_discrete(x, HDI, n, sim):
    output = [None]*sim
    for i in range(sim):
        draw = np.random.choice(x, size = n, replace = True)
        unique, counts = np.unique(draw, return_counts=True)
        output[i] = sort_category(unique, counts,4)
    lower = np.ones(len(output[0]))
    upper = np.ones(len(output[0]))
    output = np.transpose(output) # each element is now all the samples of one category
    for i in range(len(output)):
        lower[i] = np.quantile(output[i], q = (1-HDI)/2)
        upper[i] = np.quantile(output[i], q = 1-(1-HDI)/2)
    return [lower, upper]

def compute_ci_continuous(x, HDI, n, sim):
    output = np.ones(sim)
    for i in range(sim):
        draw = np.random.choice(x, size = n, replace = True)
        output[i] = np.mean(draw)
    lower = np.quantile(output, q = (1-HDI)/2)
    upper = np.quantile(output, q = 1-(1-HDI)/2)
    return [lower, upper]

def computeA(m, s):
    return (m*s)/(1-m)

# Computational models

## Data preprocessing

In [ ]:
data = pd.read_csv('../cog-sci-present-analysis/multiple-outcomes-scenario/ecl_dat.csv', encoding="ISO-8859-1")
emotion = data.loc[data['condition'] == 'condition 1']
noemotion = data.loc[data['condition'] == 'condition 2']

### Normalization of data

In [ ]:
human_cause_purple_emotion_data = (np.array(emotion["relationBlueOne"]) - 1) / 8.0
human_cause_red_emotion_data = (np.array(emotion["relationBlueTwo"]) - 1) / 8.0
human_cause_purple_noemotion_data = (np.array(noemotion["relationBlueOne"]) - 1) / 8.0
human_cause_red_noemotion_data = (np.array(noemotion["relationBlueTwo"]) - 1) / 8.0

human_belief_purple_emotion_data = (np.array(emotion["exp1"]) - 1) / 8.0
human_belief_red_emotion_data = (np.array(emotion["exp2"]) - 1) / 8.0
human_belief_purple_noemotion_data = (np.array(noemotion["exp1"]) - 1) / 8.0
human_belief_red_noemotion_data = (np.array(noemotion["exp2"]) - 1) / 8.0

human_desire_purple_emotion_data = (np.array(emotion["desire1"]) - 1) / 8.0
human_desire_red_emotion_data = (np.array(emotion["desire2"]) - 1) / 8.0
human_desire_purple_noemotion_data = (np.array(noemotion["desire1"]) - 1) / 8.0
human_desire_red_noemotion_data = (np.array(noemotion["desire2"]) - 1) / 8.0

human_knowledge_emotion_data = (np.array(emotion["knowledge1"]) - 1) / 8.0
human_knowledge_noemotion_data = (np.array(noemotion["knowledge1"]) - 1) / 8.0

### Computing means for causal, belief, desire, and knowledge inference

In [ ]:
human_cause_purple_emotion = np.mean(human_cause_purple_emotion_data)
human_cause_red_emotion = np.mean(human_cause_red_emotion_data)
human_cause_purple_noemotion = np.mean(human_cause_purple_noemotion_data)
human_cause_red_noemotion = np.mean(human_cause_red_noemotion_data)

human_belief_purple_emotion = np.mean(human_belief_purple_emotion_data)
human_belief_red_emotion = np.mean(human_belief_red_emotion_data )
human_belief_purple_noemotion = np.mean(human_belief_purple_noemotion_data)
human_belief_red_noemotion = np.mean(human_belief_red_noemotion_data)

human_desire_purple_emotion = np.mean(human_desire_purple_emotion_data)
human_desire_red_emotion = np.mean(human_desire_red_emotion_data)
human_desire_purple_noemotion = np.mean(human_desire_purple_noemotion_data)
human_desire_red_noemotion = np.mean(human_desire_red_noemotion_data)

human_knowledge_emotion = np.mean(human_knowledge_emotion_data )
human_knowledge_noemotion = np.mean(human_knowledge_noemotion_data)

## Model (for the condition with no emotional display)

In [ ]:
# These parameters are fixed across both conditions (emotions and no-emotions)

# The model assumes that people perceive others to have high knowledgability.
# We simulated this using a beta distribution with a = 5 and b = 2 (mean of .71).
a = 5
b = 2

In [ ]:
np.random.seed(123456)

In [ ]:
with pm.Model() as model1:
    world1 = pm.Beta("world1", .1, .1, shape=(1)) # inference that the box causes the purple bulb to switch on; the U-shaped beta distribution reduces the chance of "vague" inferences
    world2 = pm.Beta("world2", .1, .1, shape=(1)) # inference that the box causes the red bulb to switch on
    knowledge_prior_p = pm.Beta("knowledge_prior_p", a, b, shape=(1))
    knowledge_prior = pm.Binomial("knowledge_prior", 1, knowledge_prior_p, shape=(1))
    knowledge_p = pm.Beta("knowledge_p", a, b, shape=(1))
    knowledge = pm.Binomial("knowledge", 1, knowledge_p, shape=(1))
    
    belief1_random = pm.Beta("belief1_random", .1, .1, shape=(1)) # belief inference that the agent thinks that the box causes the purple bulb to switch on
    belief1 = pm.Deterministic("belief1", knowledge*world1 + (1-knowledge)*belief1_random)
    belief2_random = pm.Beta("belief2_random", .1, .1, shape=(1)) # belief inference that the agent thinks that the box causes the red bulb to switch on
    belief2 = pm.Deterministic("belief2", knowledge*world2 + (1-knowledge)*belief2_random)
    
    desire = pm.Categorical("desire", [5,5,1,1], shape=(1)) # purple, red, both, others
    # Note: this prior assumes that people are more likely to desire one light bulb to turn on compared to multiple light bulbs or other alternative desires
    
    action_others_p = .05 # low prior probability of push box action
    action_p = pm.Deterministic("action_p", belief1*tt.eq(desire,0) + \ # if believe box -> purple and desire purple bulb to light up
                                belief2*tt.eq(desire,1) + \ # if believe box -> red and desire red bulb to light up
                                belief1*belief2*tt.eq(desire,2) + \ # if believe box -> both and desire both bulbs to light up
                                action_others_p*tt.eq(desire,3) ) # if believe box -> other event and desire some other event to occur
    action = pm.Binomial("action", 1, action_p, observed = [1])
    
    outcome1_p = pm.Deterministic("outcome1_p", action*world1) # whether purple bulb lights up
    outcome2_p = pm.Deterministic("outcome2_p", action*world2) # whether red bulb lights up
    outcome1 = pm.Binomial("outcome1", 1, outcome1_p, observed = [0])
    outcome2 = pm.Binomial("outcome2", 1, outcome2_p, observed = [1])
    
    # agent is happy if the outcome is congruent with his desires
    happy = outcome1*tt.eq(desire,0) + outcome2*tt.eq(desire,1) + outcome1*outcome2*tt.eq(desire,2)
    # agent is frustrated if the outcome does not satisfy his desires
    frustrated = tt.neq(outcome1,1)*tt.eq(desire,0)*tt.gt(belief1,.5) + tt.neq(outcome2,1)*tt.eq(desire,1)*tt.gt(belief2,.5) + tt.gt(belief1,.5)*tt.gt(belief2,.5)*tt.eq(desire,2)*tt.or_( tt.neq(outcome1,1), tt.neq(outcome2,1) )
    neutral = [1]
    emotion_array = tt.transpose(tt.stack(happy, frustrated, neutral))

    expression = pm.Categorical("expression",emotion_array,observed=[2] )
    draw = 2000 
    trace1 = pm.sample(draw, tune=1000, chains=4, return_inferencedata=False) 

# note: we compute the RMSE for reference (also useful to compare between models)
# Conceptually, however, it might be difficult to compare these predictions one-to-one as normalized human ratings is not the same as probability
world1posterior = np.mean(trace1["world1"].T[0])
print(rmse(world1posterior, human_cause_purple_noemotion))
world2posterior = np.mean(trace1["world2"].T[0])
print(rmse(world2posterior, human_cause_red_noemotion))

belief1posterior = np.mean(trace1["belief1"].T[0])
print(rmse(belief1posterior, human_belief_purple_noemotion))
belief2posterior = np.mean(trace1["belief2"].T[0])
print(rmse(belief2posterior, human_belief_red_noemotion))

unique, counts = np.unique(trace1["desire"], return_counts=True)
desire_posterior = sort_category(unique, counts,n=4)
desire1posterior = desire_posterior[0] + desire_posterior[2]/2
print(rmse(desire1posterior,human_desire_purple_noemotion))
desire2posterior = desire_posterior[1] + desire_posterior[2]/2
print(rmse(desire2posterior,human_desire_red_noemotion))                            
knowledge1posterior = np.mean(trace1["knowledge_p"].T[0])
print(rmse(knowledge1posterior, human_knowledge_noemotion))

### Summary statistics

In [ ]:
print("world1 posterior: " + str(np.mean(trace1["world1"])))
print("world2 posterior: " + str(np.mean(trace1["world2"])))
print("knowledge prior p: " + str(np.mean(trace1["knowledge_prior_p"])))
print("knowledge posterior p: " + str(np.mean(trace1["knowledge_p"])))
print("knowledge prior: " + str(np.mean(trace1["knowledge_prior"])))
print("knowledge posterior: " + str(np.mean(trace1["knowledge"])))
print("belief1: " + str(np.mean(trace1["belief1"])))
print("belief2: " + str(np.mean(trace1["belief2"])))
unique, counts = np.unique(trace1["desire"], return_counts=True)
print("desire: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts, 4))

### Confidence intervals

In [ ]:
np.random.seed(123456)
print("world1 :" + str(compute_ci_continuous(flatten(trace1["world1"]), HDI = .95, n = 56, sim = 1000))) 
print("world2 :" + str(compute_ci_continuous(flatten(trace1["world2"]), HDI = .95, n = 56, sim = 1000))) 
print("belief1 :" + str(compute_ci_continuous(flatten(trace1["belief1"]), HDI = .95, n = 56, sim = 1000))) 
print("belief2 :" + str(compute_ci_continuous(flatten(trace1["belief2"]), HDI = .95, n = 56, sim = 1000))) 
print("desire :" + str(compute_ci_discrete(flatten(trace1["desire"]), HDI = .95, n = 1000, sim = 1000)))
print("knowledge :" + str(compute_ci_continuous(flatten(trace1["knowledge_p"]), HDI = .95, n =56, sim = 1000))) 

In [ ]:
az.plot_trace(trace1)

## Model (for the condition with emotional displays)

In [ ]:
np.random.seed(123456)

In [ ]:
with pm.Model() as model2:
    world1 = pm.Beta("world1", .1, .1, shape=(1)) # inference that the box causes the purple bulb to switch on; the U-shaped beta distribution reduces the chance of "vague" inferences
    world2 = pm.Beta("world2", .1, .1, shape=(1)) # inference that the box causes the red bulb to switch on
    knowledge_prior_p = pm.Beta("knowledge_prior_p", 5, 2, shape=(1))
    knowledge_prior = pm.Binomial("knowledge_prior", 1, knowledge_prior_p, shape=(1))
    knowledge_p = pm.Beta("knowledge_p", 5, 2, shape=(1))
    knowledge = pm.Binomial("knowledge", 1, knowledge_p, shape=(1))
    
    belief1_random = pm.Beta("belief1_random", .1, .1, shape=(1)) # belief inference that the agent thinks that the box causes the purple bulb to switch on
    belief1 = pm.Deterministic("belief1", knowledge*world1 + (1-knowledge)*belief1_random)
    belief2_random = pm.Beta("belief2_random", .1, .1, shape=(1)) # belief inference that the agent thinks that the box causes the red bulb to switch on
    belief2 = pm.Deterministic("belief2", knowledge*world2 + (1-knowledge)*belief2_random)
    
    desire = pm.Categorical("desire", [5,5,1,1], shape=(1)) # purple, red, both, others
    # Note: this prior assumes that people are more likely to desire one light bulb to turn on compared to multiple light bulbs or other alternative desires
    
    action_others_p = .05 # low prior probability of push box action
    action_p = pm.Deterministic("action_p", belief1*tt.eq(desire,0) + \ # if believe box -> purple and desire purple bulb to light up
                                belief2*tt.eq(desire,1) + \ # if believe box -> red and desire red bulb to light up
                                belief1*belief2*tt.eq(desire,2) + \ # if believe box -> both and desire both bulbs to light up
                                action_others_p*tt.eq(desire,3) ) # if believe box -> other event and desire some other event to occur
    action = pm.Binomial("action", 1, action_p, observed = [1])
    
    outcome1_p = pm.Deterministic("outcome1_p", action*world1) # whether purple bulb lights up
    outcome2_p = pm.Deterministic("outcome2_p", action*world2) # whether red bulb lights up
    outcome1 = pm.Binomial("outcome1", 1, outcome1_p, observed = [0])
    outcome2 = pm.Binomial("outcome2", 1, outcome2_p, observed = [1])
    
    # agent is happy if the outcome is congruent with his desires
    happy = outcome1*tt.eq(desire,0) + outcome2*tt.eq(desire,1) + outcome1*outcome2*tt.eq(desire,2) 
    # agent is frustrated if the outcome does not satisfy his desires
    frustrated = tt.neq(outcome1,1)*tt.eq(desire,0)*tt.gt(belief1,.5) + tt.neq(outcome2,1)*tt.eq(desire,1)*tt.gt(belief2,.5) + tt.gt(belief1,.5)*tt.gt(belief2,.5)*tt.eq(desire,2)*tt.or_( tt.neq(outcome1,1), tt.neq(outcome2,1) )
    neutral = [1]
    emotion_array = tt.transpose(tt.stack(perturb(happy), perturb(frustrated), perturb(neutral)))
    
    expression = pm.Categorical("expression",emotion_array,observed=[1] )
    draw = 2000 
    trace2 = pm.sample(draw, tune=1000, chains=4, return_inferencedata=False) 

# note: we compute the RMSE for reference (also useful to compare between models)
# Conceptually, however, it might be difficult to compare these predictions one-to-one as normalized human ratings is not the same as probability
world1posterior = np.mean(trace2["world1"].T[0])
print(rmse(world1posterior, human_cause_purple_emotion))
world2posterior = np.mean(trace2["world2"].T[0])
print(rmse(world2posterior, human_cause_red_emotion))

belief1posterior = np.mean(trace2["belief1"].T[0])
print(rmse(belief1posterior, human_belief_purple_emotion))
belief2posterior = np.mean(trace2["belief2"].T[0])
print(rmse(belief2posterior, human_belief_red_emotion))

unique, counts = np.unique(trace2["desire"], return_counts=True)
desire_posterior = sort_category(unique, counts,n=4)
desire1posterior = desire_posterior[0] + desire_posterior[2]/2
print(rmse(desire1posterior,human_desire_purple_emotion))
desire2posterior = desire_posterior[1] + desire_posterior[2]/2
print(rmse(desire2posterior,human_desire_red_emotion))                            
knowledge1posterior = np.mean(trace2["knowledge_p"].T[0])
print(rmse(knowledge1posterior, human_knowledge_emotion))

### Summary statistics

In [ ]:
print("world1 posterior: " + str(np.mean(trace2["world1"])))
print("world2 posterior: " + str(np.mean(trace2["world2"])))
print("knowledge prior p: " + str(np.mean(trace2["knowledge_prior_p"])))
print("knowledge posterior p: " + str(np.mean(trace2["knowledge_p"])))
print("knowledge prior: " + str(np.mean(trace2["knowledge_prior"])))
print("knowledge posterior: " + str(np.mean(trace2["knowledge"])))
print("belief1: " + str(np.mean(trace2["belief1"])))
print("belief2: " + str(np.mean(trace2["belief2"])))
unique, counts = np.unique(trace2["desire"], return_counts=True)
print("desire: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts, 4))

### Confidence intervals

In [ ]:
np.random.seed(123456)
print("world1 :" + str(compute_ci_continuous(flatten(trace2["world1"]), HDI = .95, n = 68, sim = 1000))) 
print("world2 :" + str(compute_ci_continuous(flatten(trace2["world2"]), HDI = .95, n = 68, sim = 1000))) 
print("belief1 :" + str(compute_ci_continuous(flatten(trace2["belief1"]), HDI = .95, n = 68, sim = 1000))) 
print("belief2 :" + str(compute_ci_continuous(flatten(trace2["belief2"]), HDI = .95, n = 68, sim = 1000))) 
print("desire :" + str(compute_ci_discrete(flatten(trace2["desire"]), HDI = .95, n = 1000, sim = 1000)))
print("knowledge :" + str(compute_ci_continuous(flatten(trace2["knowledge_p"]), HDI = .95, n = 68, sim = 1000))) 

In [ ]:
az.plot_trace(trace2)